In [ ]:
# Import libraries
import os
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

from PIL import Image

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Directories of dataset
dataset_dir = "C:/Users/hedak/PycharmProjects/NooteboomAI/func-imagerender-prod-001/dataset"
image_size = (1920, 1920)
batch_size = 32
num_classes = 3
ds_size = 2187
train_split = 0.8
val_split = 0.1
test_split = 0.1

In [ ]:
data_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size)

In [ ]:
data_ds = data_ds.shuffle(10000, seed=12)

In [ ]:
train_size = int(train_split * ds_size)
val_size = int(val_split * ds_size)
test_size = int(test_split * ds_size)

train_ds = data_ds.take(train_size)    
val_ds = data_ds.skip(train_size).take(val_size)
test_ds = data_ds.skip(train_size).skip(test_size)

In [ ]:
plt.figure(figsize=(10, 10))

for images, labels in data_ds.take(1):
    for i in range(32):
        ax = plt.subplot(6, 6, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(labels[i])
        plt.axis("off")

In [ ]:
resizing_augmentation = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(224, 224),
    ],
)

In [ ]:
inputs = layers.Input(shape=(1920, 1920, 3))
x = resizing_augmentation(inputs)
outputs = EfficientNetB0(include_top=True, weights=None, classes=num_classes)(x)

model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.summary()

In [ ]:
epochs = 40  # @param {type: "slider", min:10, max:100}
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)

In [ ]:
results = model.evaluate(test_ds, batch_size=batch_size)